In [2]:
import numpy as np
import pandas

import csv


In [3]:
brand_mapping = {
    'name': 'Brand',
    'supplier': u'Supplier'
}
nls_brand_mapping = {
    'name': u'Бренд',
    'supplier': u'Поставщик'
}
product_mapping = {
    'article': u'Артикул',
    'country': 'Country',
    'collection': 'Collection',
    'wood_id': u'Порода дерева',
    'name': u'Name/ Wood',
    'additional_info': 'Additional characteristics',
    'color_id': u'Цвет (из 9 вариантов)',
    'color2_id': u'Цвет (из 9 вариантов).1',
    'grading_id': u'Селекция',
    'thickness': u'Толщина',
    'width': u'Ширина',
    'length': u'Длина',
    'texture': 'Texture',
    'layer': 'Bottom layer/ Middle layer (for Admonter panels)',
    'installation': 'Installation',
    'surface': 'Surface',
    'construction_id': u'Тип конструкции',
    'units': 'Units',
    'packing_type': 'Packing Type',
    'weight': u'Вес 1 единицы измерения',
    'amount': u'Количество товара в упаковке (в единицах измерения)',
    'purchase_price': u'Закупочная цена',
    'currency': u'Валюта',
    'suppliers_discount': u'Скидка от производителя',
    'margin': u'Коэффициент наценки',
    'pattern_id': u'Паттерн',
    'category_id': u'Категория'
}
nls_product_mapping = {
    'country': u'Страна',
    'collection': u'Коллекция',
    'name': u'Название/ Порода дерева',
    'additional_info': u'Дополнительные характеристики',
    'texture': u'Текстура поверхности',
    'layer': u'Нижний слой/ Средний слой (для панелей Admonter)',
    'installation': u'Способ укладки',
    'surface': u'Покрытие поверхности',
    'units': u'Единица измерения',
    'packing_type': u'Тип упаковки'
}

In [4]:
def get(row, col, errors):
    if col not in row:
        errors.update([col + " is not in dataset"])
    else:
        value = row[col]
        if unicode(value) == 'nan':
            return "NULL"
        if isinstance(value, basestring):
            return value.strip()
        elif isinstance(value, float) and value.is_integer():
            return int(value)
        else:
            return value

In [5]:
products, nls_products, brands, nls_brands, photos = [], [], [], [], []

ids = {
    'product': 1,
    'brand': 1,
    'photo': 1
}

def brand_id(row, errors):
    en, ru = get(row, u'Brand', []), get(row, u'Бренд', [])
    matched = [brand for brand in brands if brand['name'] == en]
    if len(matched) > 0:
        return matched[0]['brand_id']
    else:
        brand = {
            'brand_id': ids['brand'],
        }
        for col, col_name in brand_mapping.items():
            brand[col] = get(row, col_name, errors)
        brands.append(brand)
        
        nls_brand = {
            'brand_id': ids['brand'],
            'language_id': 2,
        }
        for col, col_name in nls_brand_mapping.items():
            nls_brand[col] = get(row, col_name, errors)
        nls_brands.append(nls_brand)
        
        ids['brand'] += 1
        return ids['brand'] - 1
            
def read_file(name, df):
    errors = set()
    df.columns = [unicode(df.iloc[0, idx]).strip() if col.startswith("Unnamed") else unicode(col).strip() for idx, col in enumerate(df.columns)]
    for _, row in df.iterrows():
        if unicode(get(row, u'Brand', errors)) != 'NULL' and unicode(get(row, u'Бренд', errors)) != 'NULL':
            product = {
                'product_id': ids['product'],
                'brand_id': brand_id(row, errors),
                'sheet': name.strip(),
                'status': 0
            }
            for col, col_name in product_mapping.items():
                product[col] = get(row, col_name, errors)
            products.append(product)
            
            nls_product = {
                'product_id': ids['product'],
                'language_id': 2
            }
            for col, col_name in nls_product_mapping.items():
                nls_product[col] = get(row, col_name, errors)
            nls_products.append(nls_product)
            
            for photo_key in [key for key in row.keys() if key.startswith(u'Фото')]:
                if get(row, photo_key, errors) != u'#' and unicode(get(row, photo_key, errors)) != 'NULL':
                    if photo_key.endswith(u"(А)"):
                        letter = u'A'
                    elif photo_key.endswith(u"(Б)"):
                        letter = u'B'
                    elif photo_key.endswith(u"(В)"):
                        letter = u'C'
                    else:
                        letter = u''
                    for photo_name in unicode(get(row, photo_key, errors)).split(', '):
                        if photo_name.endswith(u"А"):
                            photo_name = photo_name[:-1]
                            letter = u'A'
                        elif photo_name.endswith(u"Б"):
                            photo_name = photo_name[:-1]
                            letter = u'B'
                        elif photo_name.endswith(u"В"):
                            photo_name = photo_name[:-1]
                            letter = u'C'
                        elif photo_name.endswith(u"Г"):
                            photo_name = photo_name[:-1]
                            letter = u'G'
                        elif photo_name.endswith(u"Д"):
                            photo_name = photo_name[:-1]
                            letter = u'D'
                        elif photo_name.endswith(u"Е"):
                            photo_name = photo_name[:-1]
                            letter = u'E'
                        photos.append({
                            'photo_id': ids['photo'],
                            'product_id': ids['product'],
                            'name': get(row, 'Brand', errors) + " - " + photo_name + letter,
                            'description': photo_key
                        })
                        ids['photo'] += 1
                    
            ids['product'] += 1
            
    print (name, 'sum:', len(products), "\n", errors)
    return products, nls_products, photos

In [ ]:
def ensureUtf(s):
  try:
      if type(s) == unicode:
        return s.encode('utf8', 'ignore')
  except: 
    return str(s)

sheets = pandas.read_excel(u"../raw data/БАЗА_02-08 (1).xlsx", sheetname=None)

for name, sheet in sheets.items():
    results = read_file(name, sheet)

In [12]:
import warnings
warnings.filterwarnings('error')

In [20]:
df_products = pandas.DataFrame(products)
df_products.index = df_products['product_id']
df_products[[
    'article', 'name', 'category_id', 'brand_id', 'country', 'collection', 'wood_id', 'additional_info', 'color_id', 'color2_id', 'grading_id',
    'thickness', 'width', 'length', 'texture', 'layer', 'installation', 'surface', 'construction_id', 'units', 'packing_type',
    'weight', 'amount', 'purchase_price', 'purchase_price', 'currency', 'suppliers_discount', 'margin', 'pattern_id', 'sheet', 'status'    
]].to_csv("output_products.csv", encoding="utf-8", header=False, quoting=csv.QUOTE_ALL)

KeyError: 'product_id'

In [16]:
df_brands = pandas.DataFrame(brands)
df_brands.index = df_brands['brand_id']
df_brands[[
        'name', 'supplier'
]].to_csv("output_brands.csv", encoding="utf-8", header=False, quoting=csv.QUOTE_ALL)

KeyError: 'brand_id'

In [ ]:
df_nls_brands = pandas.DataFrame(nls_brands)
df_nls_brands.index = df_nls_brands['brand_id']
df_nls_brands[[
        'language_id', 'name', 'supplier'
]].to_csv("output_nls_brands.csv", encoding="utf-8", header=False, quoting=csv.QUOTE_ALL)

In [ ]:
df_photos = pandas.DataFrame(photos)
df_photos.index = df_photos['photo_id']
df_photos[[
        'product_id', 'name', 'description'
]].to_csv("output_photos.csv", encoding="utf-8", header=False, quoting=csv.QUOTE_ALL)